# Training generative models

In this notebook, we are going to showcase quickly how to train large generative models.

The goal of this lab is also to demonstrate that there exists a lot of pre-built methods that can automate common tasks.

## Download requirements

First install a library necessary to run quantized models.

In [ ]:
pip install -U accelerate==1.8.1 peft==0.15.0 bitsandbytes==0.46.0 transformers==4.52.4 datasets

## Load a generative model

We are going to load a generative model. Here we make use of heavy optimization technique to make sure everything fits on Collab GPU.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_id = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"

from transformers import BitsAndBytesConfig


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=False,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id, quantization_config=bnb_config, device_map="auto"
)


In [ ]:
print(model)

🚧 **TODO** 🚧

Experiment with the generation of the model.

In [ ]:
text = "Paris is the capital of"
DEVICE = "cuda:0"


inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
outputs = model.generate(**inputs, max_new_tokens=200, temperature=1.0, do_sample=True)
print(tokenizer.decode(outputs[0]))

## Training

First load an instruction dataset.

In [ ]:
from datasets import load_dataset


def process_data(sample):
    return tokenizer(sample["text"])


data = load_dataset(
    "tatsu-lab/alpaca",
)
data = data.map(process_data, batched=True)

In [ ]:
data

In [ ]:
print(data["train"][0]["text"])

Since this model is quite big, we are going to reduce the number of trainable parameters.

To do that, we use **Parameter Efficient Fine-Tuning** (PEFT).

In PEFT, we do not tune the full model. Only a small subsets of the parameters are trained, while the others are frozen (i.e., not updated during training).

🚧 **TODO** 🚧

Explain why PEFT reduces the memory cost.


In [ ]:
from peft import prepare_model_for_kbit_training

# This determines automatically which module can be used for quantized training inside the model
model = prepare_model_for_kbit_training(model)

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

The cell below adds a small amount of extra-paramter to the model using Lora technique, that is described here: https://arxiv.org/abs/2106.09685

In [ ]:
from peft import LoraConfig, get_peft_model

# Use default config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

We now use the Trainer class of HuggingFace. On simple installations it can be very effective.

In [ ]:
import transformers

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        report_to="none",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False
trainer.train()

In [ ]:
text = "### Instruction:\nPropose an outdoor activity.\n\n### Response:\n"
inputs = tokenizer(text, return_tensors="pt").to(DEVICE)
model.config.use_cache = True

outputs = model.generate(**inputs, max_new_tokens=200, temperature=1.0, do_sample=True)
print(tokenizer.decode(outputs[0]))